## Import CBC SDK
Check to see if the CBC SDK is installed. If it isn't, install it.

In [ ]:
try:
  from cbc_sdk.rest_api import CBCloudAPI
  print('⬢ carbon-black-cloud-sdk is installed\n')

except:
    print('⬢ carbon-black-cloud-sdk is missing. Installing the carbon-black-cloud-sdk module via pip.\n')

    from pip._internal import main as pipmain
    pipmain(['install', 'carbon-black-cloud-sdk'])
    
    from cbc_sdk.rest_api import CBCloudAPI
    print('\n⬢ Installed the carbon-black-cloud-sdk module.')

## Get Basic Params
This will prompt for API credentails, device_id, and other information required for this notebook

In [ ]:
import json


url = input('Enter the URL of your Carbon Black Cloud instance: ')
org_key = input('Enter the CBC Org Key: ')
api_id = input('Enter the CBC API ID: ')
api_key = input('Enter the CBC API Key: ')
device_id = input('Enter the device_id of the endpoint you want to test with: ')

api_token = '/'.join([api_key, api_id])
cbc_sdk = CBCloudAPI(url=url, org_key=org_key, token=api_token)

# SOAR Actions: Context

## Get Alerts

See the Alerts Documentation on [Readthedocs](https://carbon-black-cloud-python-sdk.readthedocs.io/en/latest/cbc_sdk.platform/#module-cbc_sdk.platform.alerts).

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.alerts** |  | ☑️ |  |  |  |



In [ ]:
from cbc_sdk.platform import BaseAlert

def get_alerts(query="", window="-1d", silent=False):
    alert_query = cbc_sdk.select(BaseAlert).where(query).set_time_range("create_time", range=window)
    alerts = list(alert_query)
    if not silent:
      unique_device_ids = list(set([alert.device_id for alert in alerts]))
      print("Found {} alert(s) across {} device(s)".format(len(alerts), len(unique_device_ids)))
      for device_id in unique_device_ids:
          device_alerts = [alert for alert in alerts if alert.get("device_id") == device_id]
          print("\tDevice {} (ID: {}) had {} alerts with sensor actions: {} and run states: {}".format(
                  ", ".join(list(set([alert.get("device_name") for alert in device_alerts]))),
                  device_id,
                  len(device_alerts),
                  # Getting a unique set of sensor actions when the sensor_action could be missing or null
                  ", ".join(list(set([(alert.get("sensor_action", "ALLOW") or "ALLOW") for alert in device_alerts]))),
                  ", ".join(list(set([(alert.get("run_state") or "") for alert in device_alerts])))))
    return alerts

In [ ]:
query = "device_id:{0}".format(device_id)
alerts = get_alerts(query=query, window="-1h")

## Get Processes
Get all processes seen on a device within a certain time window

See the Processes documentation on [Readthedocs](https://carbon-black-cloud-python-sdk.readthedocs.io/en/latest/cbc_sdk.platform/#module-cbc_sdk.platform.processes).

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.search.events** |    ☑️   |  ☑️   |       |         |        |

In [ ]:
from cbc_sdk.platform import Process

def get_processes(device_id, window="-1h"):
    query = cbc_sdk.select(Process).where(f"device_id:{device_id}").set_time_range(window=window)
    query._default_args['fields'] = ["*", "process_cmdline"]
    print("{0:16} {1:5} {2:30} {3:60}".format("Device Name", " PID ", "User", "Process Cmdline"))
    print("{0:16} {1:5} {2:30} {3:60}".format("-"*11, "-"*5,  "-"*20, "-"*12))
    for process in list(query):
        device_name = process.get("device_name")
        process_pid = process.get("process_pid", [0])[0]
        process_username = process.get("process_username", [""])[0]
        try:
            process_cmdline = process.get("process_cmdline", [""])[0]
        except Exception as e:
            process_cmdline = ""

        print("{0:16} {1:5} {2:30} {3:60}".format(device_name,
                                                  process_pid,
                                                  process_username,
                                                  process_cmdline))

In [ ]:
get_processes(device_id, "-1h")

## Search for a process’ metadata by process GUID
Get the most up to date metadata of a specific process. Very useful for process & cmdline information following a watchlist hit.

See the [Process Details API](https://developer.carbonblack.com/reference/carbon-black-cloud/platform/latest/platform-search-api-processes/#request-details-of-processes-v2) documentation for more info.

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.search.events** | ☑️ | ☑️ |  |  |  |


In [ ]:
from cbc_sdk.platform import Process

def get_process_metadata(process_guid, fields=None):
    try:
        details = cbc_sdk.select(Process, process_guid).get_details()
        if fields:
            for field in fields:
                print("{}: {}".format(field, details.get(field)))
        else:
            print(json.dumps(details, indent=4))
        return details
    except Exception as e:
        print("Process {} not found {}".format(process_guid, e))

In [ ]:
details = get_process_metadata('7DESJ9GN-00452906-00001c18-00000000-1d79dfa2bdc9fe8', ["process_cmdline", "process_username"])
# details = get_process_metadata('7DESJ9GN-00452906-00001c18-00000000-1d79dfa2bdc9fe8')

## Search for Enriched Events 
Get enriched events matching a query.  

> **Requirements:** Endpoint Standard  

Examples:
* Find all external network connections made from a specific endpoint.
* Find all endpoints that have seen a specific registry key modification.

See the [Enriched Events Search API](https://developer.carbonblack.com/reference/carbon-black-cloud/cb-defense/latest/platform-search-api-enriched-events/) documentation for more info.

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.search.events** | ☑️ | ☑️ |  |  |  |


In [ ]:
from cbc_sdk.endpoint_standard import EnrichedEvent

def search_enriched_events(query_str, window="-1d", fields=None):
    query = cbc_sdk.select(EnrichedEvent).where(query_str).set_time_range(window=window)
    events = list(query)
    for event in events:
        event = event.get_details()
        if fields:
            for field in fields:
                print("{}: {}".format(field, event.get(field)))
            print("-"*20)
        else:
            print(event) 
    return events

In [ ]:
query = "event_type:netconn AND device_id:{0}".format(device_id)
events = search_enriched_events(query, window="-1h", 
    fields=["device_name", "netconn_domain", "event_network_remote_port", "process_cmdline"])

## Get Alerted Events
Get the enriched events associated with an Analytics alert, which includes critical alert triage information such as the process cmdline. 

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.alerts** |  | ☑️ |  |  |  |
| **org.search.events** | ☑️ | ☑️ |  |  |  |

In [ ]:
from cbc_sdk.platform import CBAnalyticsAlert
from cbc_sdk.endpoint_standard import EnrichedEvent

def get_alerted_events(alert_id, silent=False, fields=None):
    alert = cbc_sdk.select(CBAnalyticsAlert, alert_id)
    events = alert.get_events()
    if not silent:
        for event in events:
            if fields:
                for field in fields:
                    print("{}: {}".format(field, event.get(field)))
            else:
                print(event)
    return events

In [ ]:
events = get_alerted_events('4477443a2558fcdf99d11eb935515fc69969bb2')

## Get Device(s) Info

Get the latest information about devices/endpoints, including the state, OS, sensor version, and last check-in 

 - [ReadTheDocs](https://carbon-black-cloud-python-sdk.readthedocs.io/en/latest/cbc_sdk.platform/#module-cbc_sdk.platform.devices)
 - [Developer Documentation](https://developer.carbonblack.com/reference/carbon-black-cloud/platform/latest/devices-api/#search-devices) 
 - [Device Schema (available fields)](https://developer.carbonblack.com/reference/carbon-black-cloud/platform/latest/devices-api/#schemas)

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **device** |        |  ☑️   |       |         |        |


In [ ]:
from cbc_sdk.platform import Device

def get_devices(query_str=""):
    query = cbc_sdk.select(Device).where(query_str)
    query = query.sort_by('id', 'ASC')
    devices = list(query)
    print("{0:9} {1:20} {2:15} {3:18} {4:25} {5}".format("ID", "Hostname", "OS", "IP Address", "Last Checkin Time", "Status"))
    for device in devices:
        # print(vars(device))
        print("{0:9} {1:20} {2:15} {3:18} {4:25} {5:20}".format(
            device.get("id") or "-", 
            device.get("name") or "-",
            device.get("os_version") or "Unknown",
            device.get("last_internal_ip_address") or "Unknown",
            device.get("last_contact_time") or "Never",
            device.get("status") or "Unknown"))

In [ ]:
query = input('Enter search term: ')
get_devices(query)

## Download Binary (from UBS) 

Fetch a potentially malicious binary and send to a sandbox for analysis 

[ReadTheDocs](https://carbon-black-cloud-python-sdk.readthedocs.io/en/latest/cbc_sdk.enterprise_edr/#module-cbc_sdk.enterprise_edr.ubs)

In [ ]:
from cbc_sdk.enterprise_edr import Binary

def download_binary(sha256):
    binary = Binary(cbc_sdk, sha256.lower())
    download_url = binary.download_url()
    print('Download URL: {0}'.format(download_url))

In [ ]:
download_binary('80B110B91730729BE60C7D79C55FFF0EC893FD4CFB5F44D04C433EE8E95C5E20')

## Get Logged In Users
Get a list of users currently logged in to a set of endpoints
> **Requirements:** Audit & Remediation  

Examples:
* Identify who was logged in following a credential scraping alert

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.livequery.manage** | ☑️ | ☑️ |  |  |  |


In [ ]:
from cbc_sdk.audit_remediation import Run, Result
from datetime import datetime
import time

def get_logged_in_users(device_ids):		
    try:
      # Run the Live Query on only specific endpoints
      live_query = cbc_sdk.select(Run).where(sql="SELECT * FROM logged_in_users").device_ids(device_ids).name("CBC SDK: Credential Scraping (Logged In Users)")
      live_query_run = live_query.submit()	
      submit_time = datetime.utcnow()
      while live_query_run.status != "COMPLETE":
          refresh = live_query_run._refresh()
          if live_query_run.status == "COMPLETE" or (datetime.utcnow() - submit_time).seconds > 600:
              results = list(cbc_sdk.select(Result).run_id(live_query_run.id))
              matched = [result for result in results if result.status == "matched"]
              print("{0:30} {1:10} {2:10} {3:30} {4:10} {5:10} {6}".format("Device Name", "Device ID", "OS", "User", "Type", "Session", "Time"))
              for result in matched:
                  print("{0:30} {1:10} {2:10} {3:30} {4:10} {5:10} {6}".format(
                      result.device.get("name"),
                      result.device.get("id"),
                      result.device.get("os"),
                      result.fields.get("user"),
                      result.fields.get("type"),
                      result.fields.get("tty"),
                      datetime.fromtimestamp(result.fields.get("time")).isoformat()))
              break
          time.sleep(5)
          
    except Exception as e:
      print("Failed to get logged in users with error: {}".format(e))

In [ ]:
get_logged_in_users([device_id])

## Get Report Info
Gets the information about a Report including the description, link, and tags
For use after a Watchlist Alert has fired.
* Requires Enterprise EDR

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.feeds** |  | ☑️ |  |  |  |
| **org.watchlists** |  | ☑️ |  |  |  |

In [ ]:
from cbc_sdk.enterprise_edr import Watchlist, Report

def get_report_info(watchlist_id, report_id):
    watchlist = cbc_sdk.select(Watchlist, watchlist_id)
    if watchlist.classifier.get("key") == "feed_id":
        watchlist_reports = watchlist.feed.reports

    else:
        watchlist_reports = watchlist.reports

    try:
        report = [r for r in watchlist_reports if report_id == "{}-{}".format(watchlist.classifier.get("value"), r.id)]
        report = report[0]
        print("Report: {}\n\tWatchlist(s): {}\n\tSeverity: {}\n\tDescription: {}\n\tTags: {}\n\tLink: {}".format(
            report.get("title"), 
            watchlist.get("name"), 
            report.get('severity'), 
            report.get("description","None").replace("\n","\n\t\t"),
            ",".join(report.get("tags", ["None"])),
            report.get("link")))

    except:
      print("Report: (Deleted)\n\tWatchlist(s): {}".format(watchlist.get("name")))

In [ ]:
get_report_info("uxgHiAbKT2aQQlzFZWQT4Q", "FFAGQQZQRmOhg0clEA5V1g-c46f2504-fce8-4aac-836b-1fb5b4cd0997")

# SOAR Actions: Remediation

## Ban a process hash
Add a sha256 process hash to the banned list via Reputation Overrides

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.reputations** | ☑️ |  |  |  |  |


In [ ]:
from cbc_sdk.platform import ReputationOverride

def ban_hash(sha256_hash, process_name, description="Banned via CBC SDK"):
    try:
        ban = ReputationOverride.create(cbc_sdk, {
            "description": description,
            "override_list": "BLACK_LIST",
            "override_type": "SHA256",
            "sha256_hash": sha256_hash,
            "filename": process_name
        })
        print("Successfully banned sha256 hash '{}' of process '{}' with comment '{}'".format(sha256_hash, process_name, description))

    except Exception as e:
        print("Failed to ban sha256 hash '{}' of process '{}'. Error: {}".format(sha256_hash, process_name, e))

In [ ]:
ban_hash("bffd6d1920f1356a5be00c9c3aef02e2664ad50d5c40d5a21575e9000834c7ad", "rat_sakula.exe", "Malicious Sakula v1.0 RAT (banned by CBC SDK)")

## Isolate (Quarantine) device
When the endpoint is suspected to be compromised, isolate it so that the only network communication allowed is with Carbon Black Cloud.

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **device** |  | ☑️ |  |  |  |
| **device.quarantine** |  |  |  | ☑️ |  |

[ReadTheDocs](https://carbon-black-cloud-python-sdk.readthedocs.io/en/latest/cbc_sdk.platform/#cbc_sdk.platform.devices.Device.quarantine) | [API Documentation](https://developer.carbonblack.com/reference/carbon-black-cloud/platform/latest/devices-api/#quarantine)

In [ ]:
from cbc_sdk.platform import Device

def isolate_device(device_id):
    device = cbc_sdk.select(Device, device_id)
    device.quarantine(True)

In [ ]:
isolate_device(device_id)

## Move device to new Policy
Move the endpoint to a more restrictive policy (e.g. one that does not allow powershell or any unknown processes to execute) 

Move the asset to a policy that has LiveResponse enabled, required for LiveResponse actions. 

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **device** |  | ☑️ |  |  |  |
| **device.policy** |  |  | ☑️ |  |  |

[ReadTheDocs](https://carbon-black-cloud-python-sdk.readthedocs.io/en/latest/cbc_sdk.platform/#cbc_sdk.platform.devices.Device.policy_id) | [API Documentation](https://developer.carbonblack.com/reference/carbon-black-cloud/platform/latest/devices-api/#update-policy)


In [ ]:
from cbc_sdk.platform import Device

def move_device_to_policy(device_id, policy_id):
    device = cbc_sdk.select(Device, device_id)
    device.update_policy(policy_id)

In [ ]:
move_device_to_policy(device_id, 6529)

## Live Response

Live Response is a set of APIs that allows the user to take actions on the endpoint, similar to a terminal.

### Start a Live Response Session


In [ ]:
from cbc_sdk.platform import Device

def start_session(device_id):
    device = cbc_sdk.select(Device, device_id)
    lr = device.lr_session()
    return lr

def end_session(lr_session):
    lr_session.close()

In [ ]:
lr_session = start_session(device_id)

### List running processes on an endpoint 

Identify if the malicious process is still running

See the [Live Response API](https://developer.carbonblack.com/reference/carbon-black-cloud/platform/latest/live-response-api/) documentation for more info.

---

| Permission                   | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------------------------- | :----: | :--: | :----: | :-----: | :----: |
| **org.liveresponse.session** |  ☑️    |  ☑️   |   ☑️   |         |   ☑️   |
| **org.liveresponse.process** |        |  ☑️  |        |         |        |


In [ ]:
from cbc_sdk.platform import Device

def list_running_processes(device_id, silent=False):
    device = cbc_sdk.select(Device, device_id)
    
    # The `with` statement will close the session when the task is complete
    with device.lr_session() as lr:
        running_processes = lr.list_processes()
        if not silent:
          print("{0:5} {1:60}".format(" PID", "Process Command"))
          print("{0:5} {1:60}".format('-'*5, '-'*60))
          for process in running_processes:
              print('{0:5} {1:60}'.format(process['process_pid'], process['process_cmdline']))
        return running_processes

In [ ]:
processes = list_running_processes(device_id)

### Kill a process on an endpoint
Kill a running process on the specified endpoint by name or pid.  

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.liveresponse.session** | ☑️ | ☑️ | ☑️ |  | ☑️ |
| **org.liveresponse.process** |  | ☑️ |  |  | ☑️ |
| **org.device** |  | ☑️ |  |  |  |

[ReadTheDocs](https://carbon-black-cloud-python-sdk.readthedocs.io/en/latest/cbc_sdk.platform/#cbc_sdk.platform.devices.Device.lr_session) | [API Documentation](https://developer.carbonblack.com/reference/carbon-black-cloud/platform/latest/live-response-api/#kill)



In [ ]:
from cbc_sdk.platform import Device

def kill_process_by_pid(lr_session, device_id, process_pid):
    device = cbc_sdk.select(Device, device_id)
    try:
        # with device.lr_session() as lr:
        lr_session.kill_process(process_pid)
        print("Successfully killed process pid '{}' on device '{}' (ID={})".format(process_pid, device.get("name"), device_id))

    except Exception as e:
        print("Failed to kill process pid '{}' on device '{}' (ID={}) with error: {}".format(process_pid, device.get("name"), device_id, e))

In [ ]:
kill_process_by_pid(lr_session, device_id, 3876)

In [ ]:
from cbc_sdk.platform import Device

def kill_process_by_path(device_id, process_path):
    device = cbc_sdk.select(Device, device_id)
    try:
        with device.lr_session() as lr:
            success = False
            processes = lr.list_processes()
            for process in processes:
                if process.get('process_path') == process_path:
                    lr.kill_process(process.get('process_pid'))
                    success = True
                    print("Successfully killed process '{}' pid '{}' on device '{}' (ID={})".format(process.get('process_pid'), device.get("name"), device_id))

                if not success:
                    print("Failed to kill process '{}' on device '{}' (ID={}) with error: Process not found".format(process_path, device.get("name"), device_id, e))

    except Exception as e:
        print("Failed to kill process '{}' on device '{}' (ID={}) with error: {}".format(process_path, device.get("name"), device_id, e))

In [ ]:
kill_process_by_path(device_id, "c:\\windows\\system32\\ping.exe")

### List a directory on an endpoint

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.liveresponse.file** |  | ☑️ |  |  |  |

[ReadTheDocs](https://carbon-black-cloud-python-sdk.readthedocs.io/en/latest/cbc_sdk/#cbc_sdk.live_response_api.LiveResponseSession)  | [API Documentation](https://developer.carbonblack.com/reference/carbon-black-cloud/platform/latest/live-response-api/#directory-list)

In [ ]:
from cbc_sdk.platform import Device

def list_directory(lr_session, folder_path):
    folder_path = folder_path.replace('\\', '\\\\')

    print("{0:10} {1:20} {2:5}".format('Type', 'Name', 'Size'))
    print("{0:10} {1:20} {2:5}".format('----', '----', '----'))

    response = lr_session.list_directory(folder_path)
    for item in response:
        item_type = 'DIR' if 'DIRECTORY' in item['attributes'] else 'FILE'
        print('{0:10} {1:20} {2:5}'.format(item_type,
                                            item['filename'],
                                            item['size']))

In [ ]:
list_directory(lr_session, 'C:\\Users\\John Doe\\Desktop\\')

### Put a file on an endpoint

Upload a file to the endpoint. 

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.liveresponse.file** | ☑️ |  |  |  |  |

[ReadTheDocs]() | [API Documentation]()

In [ ]:
from cbc_sdk.platform import Device

def upload_file(lr_session, file_path):
    # Create the file locally
    fp = open('test.bat', 'w')
    fp.write('hostname')
    fp.close()

    file_path = file_path.replace('\\', '\\\\')

    lr_session.put_file(open('test.bat', 'r'), file_path)

In [ ]:
result = upload_file(lr_session, 'C:\\Users\\John Doe\\Desktop\\test.bat')

### Execute a custom script on the endpoint

Upload and execute a Powershell file on a windows endpoint to gather data or update configuration. 

Can be combined with the Context “Get a file from an endpoint” if the script generates an output file. 

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.liveresponse.process** |  |  |  | ☑️ |  |

[ReadTheDocs]()  | [Documentation]()



In [ ]:
from cbc_sdk.platform import Device

def execute_script(lr_session, file_path):
    # Escape the file_path
    file_path = file_path.replace('\\', '\\\\')

    result = lr_session.create_process(file_path, wait_for_completion=True, wait_for_output=False)
    print(result)

In [ ]:
result = execute_script(lr_session, 'C:\\Users\\John Doe\\Desktop\\test.bat')

### Get a file from an endpoint 

Find and get the full contents of a file from an endpoint
Examples:
* Get the output of a forensic toolkit previous executed 
* Get a suspicious MS Office or Powershell file that executed on an endpoint 

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.liveresponse.file** |  | ☑️ |  |  |  |

In [ ]:
def get_file(lr_session, file_path):
    # Escape the file_path
    file_path = file_path.replace('\\', '\\\\')

    file = lr_session.get_file(file_path)
    return file

In [ ]:
file = get_file(lr_session, 'C:\\Users\\John Doe\\Desktop\\test.bat')

### Delete file on endpoint 

Delete a malicious file on an endpoint 

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.liveresponse.file** |  |  |  |  | ☑️ |

[ReadTheDocs]() | [Documentation](https://developer.carbonblack.com/reference/carbon-black-cloud/platform/latest/live-response-api/#delete-file-1)

In [ ]:
def delete_file(lr_session, file_path):
    # Escape the file_path
    file_path = file_path.replace('\\', '\\\\')

    lr_session.delete_file(file_path)

In [ ]:
delete_file(lr_session, 'C:\\Users\\John Doe\\Desktop\\test.bat')

# SOAR Actions: Orchestration

## Dismiss an alert (with or without a comment) 

Dismiss the alert in Carbon Black Cloud with comments; if an analyst looks in the CBC console, they will no longer see that alert and mistakenly triage it. 

[ReadTheDocs](https://carbon-black-cloud-python-sdk.readthedocs.io/en/latest/cbc_sdk.platform/#cbc_sdk.platform.alerts.BaseAlert.dismiss) | [Documentation](https://developer.carbonblack.com/reference/carbon-black-cloud/platform/latest/alerts-api/#create-workflow)

---

| Permission | CREATE | READ | UPDATE | EXECUTE | DELETE |
| ---------- | :----: | :--: | :----: | :-----: | :----: |
| **org.alerts** |  | ☑️ |  |  |  |
| **org.alerts.dismiss** |  |  |  | ☑️ |  |

In [ ]:
from cbc_sdk.platform import BaseAlert

def dismiss_alert(alert_id, comment="Dismissed by CBC SDK"):
    try:
        alert = cbc_sdk.select(BaseAlert, alert_id)
        alert.dismiss(comment=comment)
        print("Successfully dismissed alert ID '{}' (Device='{}', Device ID='{}', Reason='{}')".format(alert_id, alert.get("device_name"), alert.get("device_id"), alert.get("reason")))

    except Exception as e:
        print("Failed to dismiss alert ID '{}' with error: {}".format(alert_id, e))

In [ ]:
dismiss_alert("a31c6808-198f-48b8-8215-4f6255a7b0ea")

# End-to-End: Watchlist Alert Triage

## Get recent high severity Watchlist Alerts
Summarize the alert count and impacted devices

In [ ]:
query = "watchlist AND alert_severity:[8 TO *] AND device_id:{0}".format(device_id)
alerts = get_alerts(query, "-1d")

## Show each alert
Show the basics of each Watchlist alert - the report, watchlist, severity, device, and timestamp.

In [ ]:
print("{0:100} {1:50} {2:10} {3:30} {4}".format("Report", "Watchlist(s)", "Severity", "Device Name", "Last Updated"))
for alert in alerts: 
	# A report can belong to multiple watchlists; comma separate them
	print("{0:100} {1:50} {2:10} {3:30} {4}".format(
		alert.get('report_name'), 
		", ".join([watchlist.get("name") for watchlist in alert.watchlists]),
		alert.get('severity'), 
		alert.get('device_name'), 
		alert.get('last_update_time')))

## Add report details
Get additional context about the report, such as description and tags.

In [ ]:
for alert in alerts[0:3]: 
	# A report can belong to multiple watchlists; comma separate them
    print("Device: {} (ID={})\nIOC Hit: '{}'".format(alert.get("device_name"), alert.get("device_id"), alert.get("ioc_hit")))
    get_process_metadata(alert.get("process_guid"), ["process_name", "process_cmdline", "process_username", "parent_name", "parent_cmdline"])
    get_report_info(alert.get("watchlists")[0].get("id"), alert.get("report_id"))
    print("-"*50)

# End-to-End: Malware
Identify, investigate, and remediate a Malware incident

## Identify Malware
Search recent alerts for common TTPs associated with malware.

In [ ]:
query = "ttp:(RUN_BLACKLIST_APP OR RUN_MALWARE_APP OR DETECTED_MALWARE_APP OR MALWARE_DROP OR COMPANY_BLACKLIST OR DETECTED_BLACKLIST_APP) AND device_id:{0}".format(device_id)
alerts = get_alerts(query, window="-1d")

## Narrow down to critical alerts
Alerts where the malware ran and was not blocked (denied/terminated) are most critical to address with automation

In [ ]:
critical_alerts = [alert for alert in alerts if alert.get("run_state") == "RAN" and alert.get("sensor_action") not in ["DENY", "TERMINATE"] and alert.get("type") == "CB_ANALYTICS" ]
for alert in critical_alerts:
    print("Alert ID: {}".format(alert.get("id")))
    print("Alert Reason: {}".format(alert.get("reason")))
    print("Run State: {}".format(alert.get("run_state")))
    print("Sensor Action: {}".format(alert.get("sensor_action")))
    print("Severity: {}".format(alert.get("severity")))
    print("TTPS: {}".format(alert.get("threat_indicators")))
    print(alert)
    process_details = get_process_metadata(alert.get("threat_cause_process_guid"), fields=["device_name", "device_id", "process_cmdline", "process_username"])
    print("-"*20)

## Identify if the process is still running
For each alert, get the process details. 
Then use Live Response to identify if the process is still running, and kill it if it is.
The exception is any alert where the primary process is 'repmgr.exe', the name of the CBC sensor. This indicates that the sensor detected malware on disk, but the malware has not executed. 

In [ ]:
for alert in critical_alerts:
    if alert.get("threat_cause_actor_name") != "repmgr.exe":
        process_details = get_process_metadata(alert.get("threat_cause_process_guid"), fields=["process_name","process_pid", "process_cmdline"])    
        processes = list_running_processes(alert.get("device_id"), silent=True)
        found_process_running = False

        for process in processes:
            # Check the PID + process path to account for the case where the process_pid is re-used
            if process.get("pid") == process_details.get("process_pid")[0] and process.get("process_path") == process_details.get("process_name"):
                found_process_running = True
                break

        if found_process_running:
            lr_session = start_session(alert.get("device_id"))
            kill_process_by_pid(lr_session, alert.get("device_id"), process_details.get("process_pid")[0])

        else:
            print("Process no longer detected running")
    
        print("--")

## Remove malicious files
Identify and remove any files identified by the alert, either those dropped by the malware or the malware itself.

In [ ]:
from cbc_sdk.platform import CBAnalyticsAlert

files_to_delete = {}

for alert in critical_alerts:
    events = get_alerted_events(alert.get("id"), silent=True)
    filemod_events = [event for event in events if event.get("event_type") == "filemod"]

    for event in filemod_events:
        print("Device {} (ID={})".format(alert.get("device_name"), alert.get("device_id")))
        for field in ["filemod_name", "filemod_action", "filemod_reputation"]:
            print("{}: {}".format(field, event.get(field)))
        print("-"*20)

    # Add these files to the list of files to delete from the device
    files = [event.get("filemod_name") for event in filemod_events]

    if alert.get("device_id") not in files_to_delete:
        files_to_delete[alert.get("device_id")] = []
    files_to_delete[alert.get("device_id")] += files

    # On a device-by-device basis, delete the files
    for device_id in files_to_delete:
        files = list(set(files_to_delete[device_id]))
        print("Deleting the following files from device ID={}: {}".format(device_id, files))
        lr_session = start_session(device_id)
        
        for file in files:
            print("Deleting file '{}'".format(file))
            # delete_file(lr_session, file)
        end_session(lr_session)

# End-to-End: Credential Scraping

## Identify credential scraping
Get a list of Devices that have seen credential scraping alerts.
This can be found through CB Analytics Alerts with certain TTPs or Watchlist Alerts from specific reports. Alerts where the behavior (sensor action) is 'Allow' should be treated with higher criticality as the credential scraping was not blocked.

In [ ]:
query = ' OR '.join([
    'ttp:READ_SECURITY_DATA',
    'ttp:DUMP_PROCESS_MEMORY',
    'ttp:MITRE_T1003_OS_CREDENTIAL_DUMP',
    'report_name:"Credential Access - Mimikatz Credential Dumping Detected'
])
query = '({0}) AND device_id:{1}'.format(query, device_id)

alerts = get_alerts(query, window="-1d")

## Process Information
Get details about each process that triggered a credential scraping alert

In [ ]:
unique_process_guids = list(set([alert.get("threat_cause_process_guid") for alert in alerts]))

for process_guid in unique_process_guids:
    get_process_metadata(process_guid, fields=["device_name", "device_id", "process_cmdline", "process_username"])
    print("-"*20)

## Who was logged in?
Use Live Query to get a list of users currently logged in to the device.
If the credential scraping was successful, the passwords of these users may be compromised.

In [ ]:
unique_device_ids = list(set([alert.device_id for alert in alerts]))
get_logged_in_users(unique_device_ids)